<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/04_Transfer_learning_in_tensorflow_part1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 - Transer learning in tensorflow part 1 Feature Extraction

We've built a bunch of convolutional neural networks from scratch and they all them to be learning, however, there is still plenty of room for improvement.

However, doing this is very time consuming.

Luckily, there's a technique we can use to save time.

It's called **transfer learning**, in other words, taking the patterns (also called weights) another model has learned from another problem and using them for our own problem.

There are two main benefits to using transfer learning:

1. Can leverage an existing neural network architecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture which has **already learned** patterns on similar data to our own. This often results in achieving great results with less custom data.

What this means is, instead of hand-crafting our own neural network architectures or building them from scratch, we can utilise models which have worked for others.

And instead of training our own models from scratch on our own datasets, we can take the patterns a model has learned  from datasets such as [`ImageNet`](https://image-net.org/) (millions of images different objects) and use them as the foundation of our own. Doing this often leads to getting great results with less data.

## What we're going to cover

We're going to go through the following with TensorFlow:
- Introduce transfer learning 
- Using a smaller dataset to experiment faster (10% of training samples of 10 classes of food)
- Build a transer learning feature extraction model using Tensorflow Hub
- Introduce the TensorBoard callback to track model training results
- Compare model results using TensorBoard

## Using a GPU

To begin, let's check to see if we're using a GPU. Using a GPU will make sure our model trains faster than using just a CPU.


In [1]:
!nvidia-smi

Sat Aug 20 14:19:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading and becoming one with the data

In [4]:
# Get data (10% of 10 food classes from Food101)
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip


# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2022-08-20 14:23:41--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 142.250.1.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   268MB/s    in 0.6s    

2022-08-20 14:23:42 (268 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [11]:
# How many images in each Folder? Inspect our data
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories ({dirnames}), {len(filenames)} images in '{dirpath}'")

There are 2 directories (['test', 'train']), 0 images in '10_food_classes_10_percent'
There are 10 directories (['steak', 'grilled_salmon', 'fried_rice', 'chicken_wings', 'sushi', 'hamburger', 'chicken_curry', 'pizza', 'ramen', 'ice_cream']), 0 images in '10_food_classes_10_percent/test'
There are 0 directories ([]), 250 images in '10_food_classes_10_percent/test/steak'
There are 0 directories ([]), 250 images in '10_food_classes_10_percent/test/grilled_salmon'
There are 0 directories ([]), 250 images in '10_food_classes_10_percent/test/fried_rice'
There are 0 directories ([]), 250 images in '10_food_classes_10_percent/test/chicken_wings'
There are 0 directories ([]), 250 images in '10_food_classes_10_percent/test/sushi'
There are 0 directories ([]), 250 images in '10_food_classes_10_percent/test/hamburger'
There are 0 directories ([]), 250 images in '10_food_classes_10_percent/test/chicken_curry'
There are 0 directories ([]), 250 images in '10_food_classes_10_percent/test/pizza'
There

Notice how each of the training directories now has 75 images rather than 750 images. This is key to demonstrating how well transfer learning can perform with less labelled images.

The test directories still have the same amount of images. This means we'll be training on less data but evaluating our models on the same amount of test data.